In [30]:
import warnings
import pandas 
pandas.set_option('mode.use_inf_as_na', True)

warnings.filterwarnings("ignore")
warnings.simplefilter("ignore")

# This is required to catch warnings when the multiprocessing module is used
import os

os.environ["PYTHONWARNINGS"] = "ignore"
# import pertpy as pt
import scanpy as sc

import pertpy as pt
adata = pt.dt.kang_2018()

In [8]:
# adata.var_keys.

In [9]:
# result = []
# json = []
# for row in adata.obs.values:
#     result.append([item for kev, item in enumerate(row) ])
# for row in result: json.append({k: v for k, v in zip(adata.obs.columns.tolist(), row)})

# print(json)
#adata.var.highly_variable.keys

AttributeError: 'DataFrame' object has no attribute 'highly_variable'

In [32]:
adata.var

,name
index,
AL627309.1,AL627309.1
RP11-206L10.2,RP11-206L10.2
RP11-206L10.9,RP11-206L10.9
FAM87B,FAM87B
LINC00115,LINC00115
...,...
C21orf58,C21orf58
PCNT,PCNT
DIP2A,DIP2A


Index(['AL627309.1', 'RP11-206L10.2', 'RP11-206L10.9', 'FAM87B', 'LINC00115',
       'FAM41C', 'NOC2L', 'KLHL17', 'PLEKHN1', 'RP11-54O7.17',
       ...
       'LSS', 'MCM3AP-AS1', 'MCM3AP', 'AP001469.9', 'YBEY', 'C21orf58', 'PCNT',
       'DIP2A', 'S100B', 'PRMT2'],
      dtype='object', name='index', length=15706)

In [58]:

sc.pp.log1p(adata)
sc.pp.highly_variable_genes(adata)

#hv_genes = filtered_keys = df[df['column_name'] == True].index.tolist()

#normal_genes = filtered_keys = df[df['column_name'] == False].index.tolist()

#print(hv_genes, normal_genes)
hv = (list(adata.var[adata.var['highly_variable'] == True].index))
genes = (list(adata.var_names))

hv_columns = [i for i,val in enumerate(genes) if val in hv]
genes

['AL627309.1',
 'RP11-206L10.2',
 'RP11-206L10.9',
 'FAM87B',
 'LINC00115',
 'FAM41C',
 'NOC2L',
 'KLHL17',
 'PLEKHN1',
 'RP11-54O7.17',
 'HES4',
 'RP11-54O7.11',
 'ISG15',
 'AGRN',
 'C1orf159',
 'TTLL10',
 'TNFRSF18',
 'TNFRSF4',
 'SDF4',
 'B3GALT6',
 'FAM132A',
 'UBE2J2',
 'ACAP3',
 'PUSL1',
 'CPSF3L',
 'GLTPD1',
 'DVL1',
 'AURKAIP1',
 'CCNL2',
 'RP4-758J18.2',
 'MRPL20',
 'ATAD3C',
 'ATAD3B',
 'ATAD3A',
 'SSU72',
 'AL645728.1',
 'RP11-345P4.9',
 'MIB2',
 'MMP23B',
 'CDK11B',
 'RP11-345P4.10',
 'SLC35E2B',
 'CDK11A',
 'SLC35E2',
 'NADK',
 'GNB1',
 'RP1-140A9.1',
 'TMEM52',
 'PRKCZ',
 'RP5-892K4.1',
 'C1orf86',
 'AL590822.2',
 'SKI',
 'MORN1',
 'RER1',
 'PEX10',
 'PANK4',
 'RP3-395M20.12',
 'TNFRSF14',
 'RP3-395M20.9',
 'FAM213B',
 'MMEL1',
 'TTC34',
 'MEGF6',
 'RP11-46F15.2',
 'TPRG1L',
 'WRAP73',
 'TP73-AS1',
 'SMIM1',
 'LRRC47',
 'CEP104',
 'DFFB',
 'C1orf174',
 'LINC01134',
 'NPHP4',
 'KCNAB2',
 'RPL22',
 'RNF207',
 'ICMT',
 'LINC00337',
 'GPR153',
 'ACOT7',
 'RP1-202O8.3',
 'ESPN

In [56]:
hv_columns = [i for i,val in enumerate(genes) if val in hv]
hv_columns

[10,
 12,
 234,
 279,
 305,
 363,
 428,
 506,
 522,
 681,
 715,
 718,
 719,
 845,
 1042,
 1043,
 1046,
 1048,
 1049,
 1051,
 1124,
 1163,
 1192,
 1210,
 1211,
 1219,
 1236,
 1295,
 1323,
 1330,
 1351,
 1353,
 1523,
 1668,
 1671,
 1676,
 1700,
 1837,
 1898,
 1958,
 2046,
 2052,
 2055,
 2056,
 2061,
 2097,
 2138,
 2208,
 2286,
 2302,
 2461,
 2471,
 2483,
 2623,
 2636,
 2663,
 2766,
 2829,
 2927,
 2978,
 3013,
 3024,
 3151,
 3172,
 3242,
 3252,
 3356,
 3374,
 3460,
 3590,
 3696,
 3736,
 3745,
 3778,
 3822,
 3840,
 3846,
 3847,
 3881,
 3902,
 3959,
 3981,
 3992,
 4016,
 4062,
 4156,
 4228,
 4359,
 4542,
 4610,
 4619,
 4622,
 4673,
 4674,
 4689,
 4743,
 4764,
 4796,
 4812,
 4890,
 4909,
 5039,
 5053,
 5055,
 5074,
 5075,
 5077,
 5091,
 5096,
 5113,
 5119,
 5121,
 5122,
 5123,
 5127,
 5129,
 5133,
 5136,
 5137,
 5145,
 5163,
 5180,
 5296,
 5393,
 5396,
 5487,
 5653,
 5654,
 5741,
 5814,
 5851,
 5971,
 5983,
 5994,
 6001,
 6047,
 6068,
 6108,
 6136,
 6168,
 6361,
 6363,
 6397,
 6436,
 6483,
 

In [59]:
len(hv_columns)

321

In [39]:
highly_variable_columns = []


for key, gene in enumerate(normal_genes):
    if gene in hv_genes:
        highly_variable_columns += key

In [20]:
highly_variable_columns
print(hv_genes, normal_genes)

[True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, Tru

In [12]:
#preprocess
#batchcontrol
adata

AnnData object with n_obs × n_vars = 24673 × 15706
    obs: 'nCount_RNA', 'nFeature_RNA', 'tsne1', 'tsne2', 'label', 'cluster', 'cell_type', 'replicate', 'nCount_SCT', 'nFeature_SCT', 'integrated_snn_res.0.4', 'seurat_clusters'
    var: 'name', 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'log1p', 'hvg'
    obsm: 'X_pca', 'X_umap'

In [ ]:
#clustering
#cell type annotation
#embedding
#reverse transcriptome
#differential expression

#rna velocity -> predicts the future state 
#predict the probability of unseen perturbations
#test: first data set => output list of perturbations 
# take a 2nd dataset and get list of perturbations + their probability 


#representation learning (in particular, self-supervised, multi-view, and transfer learning
#https://registry.opendata.aws/tabula-muris/#usageexamples

#see what preprocessing is needed to get better accuracy

##load data

##import premade model => output list of predicted unseen perturbation

#write custom model. get better accuracy

In [ ]:
sc.pp.calculate_qc_metrics(adata)
5

In [13]:
import requests
import time
from multiprocessing import cpu_count
from multiprocessing.pool import ThreadPool

def download_parallel(args):
    cpus = cpu_count()
    results = ThreadPool(cpus - 1).imap_unordered(download_url, args)
    for result in results:
        print('url:', result[0], 'time (s):', result[1])

In [14]:
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader

In [60]:
adata.X.drop

<24673x15706 sparse matrix of type '<class 'numpy.float32'>'
	with 14184532 stored elements in Compressed Sparse Column format>

In [61]:
gene_expressed = ["string1", "string2", "string3"]

    
strings = np.array(gene_expressed)
#adata.obs['genes_expressed'] = strings
coordinates = []
cx = adata.X
cx = cx.tocoo()

#delete non highly variable genes from matrix


for i,j,v in zip(cx.row, cx.col, cx.data):
    if i in hv_columns: coordinates += [[i,j,v]]
for coord in coordinates:
    coord[1] = adata.var_names[coord[1]]

#print(coordinates)

KeyboardInterrupt: 

In [17]:
len(coordinates)

14184532

In [ ]:
rowGeneExpression = {}

for coordinate in coordinates:
    row = coordinate[0]
    gene = coordinate[1]
    pvalue = coordinate[2]
    if not row in rowGeneExpression:
        rowGeneExpression[row] = 0
    if gene in hv_genes: rowGeneExpression[row] += pvalue
print(rowGeneExpression)

In [ ]:
from collections import defaultDict
rowGeneExpression = defaultDict(0)


for column in hv_genes:
    for row_id in range(df.shape[0]):
        rowGeneExpression[row_id] = adata.X[row_id, column]
        
rowGeneExpression

In [84]:
df = adata.obs
dependent_variables =  [rowGeneExpression[row] for row in range(df.shape[0])]
df['geneExpressionCount'] = dependent_variables
numerical_values = df.select_dtypes(include=[int, float]).values.tolist()


In [85]:
import torch.optim as optim


criterion = torch.nn.CrossEntropyLoss()
#optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)


In [86]:
from torch import tensor

t_dep = tensor(dependent_variables) # pertrubations


In [87]:
t_indep = tensor(numerical_values, dtype=torch.float)
t_indep

tensor([[  3017.0000,    877.0000,    -27.6404,     14.9666,      9.0000,   1704.0000,    711.0000,   1246.0000],
        [  2481.0000,    713.0000,    -27.4936,     28.9249,      9.0000,   1614.0000,    662.0000,   1246.0001],
        [   703.0000,    337.0000,    -10.4682,     -5.9844,      3.0000,    908.0000,    337.0000,   1246.0000],
        [  3420.0000,    850.0000,    -24.3680,     20.4293,      9.0000,   1738.0000,    653.0000,   1245.9999],
        [  3158.0000,   1111.0000,     27.9522,     24.1597,      4.0000,   1857.0000,    928.0000,   1246.0000],
        [  1869.0000,    635.0000,     -0.4702,    -25.3987,      5.0000,   1525.0000,    634.0000,   1246.0000],
        [  1142.0000,    436.0000,    -15.9062,     20.0853,      9.0000,   1157.0000,    436.0000,   1246.0000],
        ...,
        [   635.0000,    424.0000,     -6.6479,     -5.5475,      3.0000,    882.0000,    423.0000,   1246.0000],
        [  1340.0000,    480.0000,      7.7202,     33.3402,      8.0000,  

In [88]:
t_indep.shape[1]

8

In [89]:



n_coeff = t_indep.shape[1]
coeffs = torch.rand(n_coeff)-0.5
coeffs

tensor([-0.3367,  0.2222, -0.2981,  0.2982,  0.3446, -0.4820, -0.3949,  0.2829])

In [90]:
def acc(coeffs): return (val_dep.bool()==(calc_preds(coeffs, val_indep)>0.1)).float().mean()
def calc_preds(coeffs, indeps): return (indeps*coeffs).sum(axis=1)
def calc_loss(coeffs, indeps, deps): return torch.abs(calc_preds(coeffs, indeps)-deps).mean()

In [91]:
preds = (t_indep*coeffs).sum(axis=1)
loss = torch.abs(preds-t_dep).mean()
loss = calc_loss(coeffs, t_indep, t_dep)



loss = torch.abs(preds-t_dep).mean()
loss

tensor(2137.1646, dtype=torch.float64)

In [92]:
import torch, numpy as np, pandas as pd
np.set_printoptions(linewidth=140)
torch.set_printoptions(linewidth=140, sci_mode=False, edgeitems=7)
pd.set_option('display.width', 140)

In [93]:
vals,indices = t_indep.max(dim=0)
t_indep = t_indep / vals

In [94]:
t_indep*coeffs

tensor([[-0.0860,  0.0707,  0.1876,  0.1038,  0.3102, -0.2727, -0.1980,  0.2829],
        [-0.0707,  0.0575,  0.1866,  0.2007,  0.3102, -0.2583, -0.1844,  0.2829],
        [-0.0200,  0.0272,  0.0711, -0.0415,  0.1034, -0.1453, -0.0939,  0.2829],
        [-0.0975,  0.0685,  0.1654,  0.1417,  0.3102, -0.2781, -0.1819,  0.2829],
        [-0.0900,  0.0896, -0.1897,  0.1676,  0.1379, -0.2971, -0.2584,  0.2829],
        [-0.0533,  0.0512,  0.0032, -0.1762,  0.1723, -0.2440, -0.1766,  0.2829],
        [-0.0325,  0.0351,  0.1080,  0.1393,  0.3102, -0.1851, -0.1214,  0.2829],
        ...,
        [-0.0181,  0.0342,  0.0451, -0.0385,  0.1034, -0.1411, -0.1178,  0.2829],
        [-0.0382,  0.0387, -0.0524,  0.2313,  0.2757, -0.2119, -0.1337,  0.2829],
        [-0.0294,  0.0377, -0.1240,  0.0073,  0.2068, -0.1805, -0.1303,  0.2829],
        [-0.0603,  0.0660,  0.0785,  0.0179,  0.1379, -0.2671, -0.2225,  0.2829],
        [-0.0434,  0.0422, -0.1707,  0.0458,  0.2068, -0.2275, -0.1456,  0.2829],
   

In [95]:
t_indep = t_indep / vals

In [96]:
coeffs.requires_grad_()


tensor([-0.3367,  0.2222, -0.2981,  0.2982,  0.3446, -0.4820, -0.3949,  0.2829], requires_grad=True)

In [97]:
loss = calc_loss(coeffs, t_indep, t_dep)

loss.backward()

In [98]:
coeffs.grad

tensor([    -0.0000,     -0.0001,     -0.0002,     -0.0001,     -0.0645,     -0.0001,     -0.0003,     -0.0008])

In [99]:
dependent_variables

[1246.0000137984753,
 1246.0000799298286,
 1245.9999904632568,
 1245.9999099373817,
 1245.999993801117,
 1246.000037074089,
 1245.9999656677246,
 1245.9999763965607,
 1245.9999561309814,
 1246.0000751018524,
 1245.999941945076,
 1245.9999556541443,
 1245.999973475933,
 1245.9999905824661,
 1245.9999458789825,
 1245.9999914169312,
 1246.0001059174538,
 1246.0000286102295,
 1246.0000500679016,
 1245.999924659729,
 1246.0000565052032,
 1246.0000257492065,
 1245.9999425411224,
 1245.9999387264252,
 1245.9999957084656,
 1245.999970793724,
 1245.9999507665634,
 1246.0000014305115,
 1246.000014424324,
 1245.9999462366104,
 1246.0000219345093,
 1246.000038087368,
 1246.0000107884407,
 1245.9999206066132,
 1246.0000133514404,
 1246.0000476241112,
 1245.9999183416367,
 1246.0000550746918,
 1246.0000044107437,
 1246.0000610351562,
 1245.9999737739563,
 1245.9999489188194,
 1245.999942779541,
 1245.9999084472656,
 1246.0000304579735,
 1245.99995470047,
 1245.9999916553497,
 1245.9999848604202,
 12

In [100]:
independent_variables = pd.DataFrame(numerical_values)

In [101]:
from fastai.data.transforms import RandomSplitter
trn_split,val_split=RandomSplitter(seed=42)(independent_variables)

In [102]:
trn_indep,val_indep = t_indep[trn_split],t_indep[val_split]
trn_dep,val_dep = t_dep[trn_split],t_dep[val_split]
len(trn_indep),len(val_indep)

(19739, 4934)

In [103]:
def update_coeffs(coeffs, lr):
    coeffs.sub_(coeffs.grad * lr)
    coeffs.grad.zero_()

In [104]:
def one_epoch(coeffs, lr):
    loss = calc_loss(coeffs, trn_indep, trn_dep)
    loss.backward()
    with torch.no_grad(): update_coeffs(coeffs, lr)
    print(f"{loss:.3f}", end="; ")

In [105]:
def init_coeffs(): return (torch.rand(n_coeff)-0.5).requires_grad_()

In [106]:
def train_model(epochs=30, lr=0.01):
    coeffs = init_coeffs()
    for i in range(epochs): one_epoch(coeffs, lr=lr)
    return coeffs

In [107]:
coeffs = train_model(18, lr=0.2)

1246.028; 1246.028; 1246.027; 1246.026; 1246.025; 1246.024; 1246.023; 1246.023; 1246.022; 1246.021; 1246.020; 1246.019; 1246.018; 1246.018; 1246.017; 1246.016; 1246.015; 1246.014; 

In [108]:
indep_cols = ['nCount_RNA' ,                
'nFeature_RNA'    ,             
'tsne1'     ,                 
'tsne2'  ,                   
'cluster'    ,                  
'nCount_SCT'   ,              
'nFeature_SCT',
'geneExpressionCount'
]    
def show_coeffs(): return dict(zip(indep_cols, coeffs.requires_grad_(False)))
show_coeffs()

{'nCount_RNA': tensor(-0.2832),
 'nFeature_RNA': tensor(0.1566),
 'tsne1': tensor(-0.3726),
 'tsne2': tensor(-0.1476),
 'cluster': tensor(-0.2011),
 'nCount_SCT': tensor(-0.2953),
 'nFeature_SCT': tensor(-0.0597),
 'geneExpressionCount': tensor(-0.3374)}

In [109]:
numerical_values

[[3017.0,
  877.0,
  -27.6403728343912,
  14.9666293340935,
  9.0,
  1704.0,
  711.0,
  1246.0000137984753],
 [2481.0,
  713.0,
  -27.4936458839978,
  28.9248851440496,
  9.0,
  1614.0,
  662.0,
  1246.0000799298286],
 [703.0,
  337.0,
  -10.4681940980147,
  -5.98438948042782,
  3.0,
  908.0,
  337.0,
  1245.9999904632568],
 [3420.0,
  850.0,
  -24.3679974111624,
  20.4292847674974,
  9.0,
  1738.0,
  653.0,
  1245.9999099373817],
 [3158.0,
  1111.0,
  27.9521697403372,
  24.1597381598606,
  4.0,
  1857.0,
  928.0,
  1245.999993801117],
 [1869.0,
  635.0,
  -0.470236382288022,
  -25.3987091482051,
  5.0,
  1525.0,
  634.0,
  1246.000037074089],
 [1142.0,
  436.0,
  -15.9061825808314,
  20.0853153562297,
  9.0,
  1157.0,
  436.0,
  1245.9999656677246],
 [817.0,
  427.0,
  -6.74039407642886,
  -23.9329848720015,
  5.0,
  961.0,
  427.0,
  1245.9999763965607],
 [1582.0,
  558.0,
  -17.682802916336,
  21.0055756282042,
  9.0,
  1397.0,
  557.0,
  1245.9999561309814],
 [728.0,
  474.0,
  26

In [110]:

acc(coeffs) + 1

##
# predictions = tf.nn.sigmoid(predictions)
# predictions = tf.where(predictions < 0.5, 0, 1)

# print('Predictions:\n', predictions.numpy())
# print('Labels:\n', label_batch)

# pass in a h5ad file and get back 
#Predictions:
# [0 0 0 0 0 1 1 1 0 0 0 1 0 1 0 0 1 0 1 0 0 0 0 0 1 1 0 1 1 1 0 0]
# Perturbations:
# [0 0 0 0 0 1 1 1 0 0 0 1 0 1 0 0 1 0 1 0 0 0 0 0 1 0 0 1 1 1 0 0]

tensor(1.)

In [111]:
def benchmarkResults():
    pass

In [112]:
import scgen
adata.obs.rename({"label": "condition"}, axis=1, inplace=True)
adata.obs["condition"].replace({"ctrl": "control", "stim": "stimulated"}, inplace=True)






adata_t = adata[
    ~(
        (adata.obs["cell_type"] == "CD4 T cells")
        & (adata.obs["condition"] == "stimulated")
    )
].copy()

cd4t_stim = adata[
    (
        (adata.obs["cell_type"] == "CD4 T cells")
        & (adata.obs["condition"] == "stimulated")
    )
].copy()








scgen.SCGEN.setup_anndata(adata_t, batch_key="condition", labels_key="cell_type")


In [113]:
model = scgen.SCGEN(adata_t, n_hidden=800, n_latent=100, n_layers=2)

In [114]:
model.train(
    max_epochs=1, batch_size=32, early_stopping=True, early_stopping_patience=25
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 3080') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Epoch 1/1: 100%|███████████████| 1/1 [00:15<00:00, 15.11s/it, v_num=1, train_loss_step=2.74e+3, train_loss_epoch=6.1e+3]

`Trainer.fit` stopped: `max_epochs=1` reached.


Epoch 1/1: 100%|███████████████| 1/1 [00:15<00:00, 15.16s/it, v_num=1, train_loss_step=2.74e+3, train_loss_epoch=6.1e+3]


In [115]:
adata_t.obsm["scgen"] = model.get_latent_representation()

In [116]:
sc.pp.neighbors(adata_t, use_rep="scgen")
#sc.tl.umap(adata)

In [117]:
adata_t

AnnData object with n_obs × n_vars = 18995 × 15706
    obs: 'nCount_RNA', 'nFeature_RNA', 'tsne1', 'tsne2', 'condition', 'cluster', 'cell_type', 'replicate', 'nCount_SCT', 'nFeature_SCT', 'integrated_snn_res.0.4', 'seurat_clusters', 'geneExpressionCount', '_scvi_batch', '_scvi_labels'
    var: 'name'
    uns: '_scvi_uuid', '_scvi_manager_uuid', 'neighbors'
    obsm: 'X_pca', 'X_umap', 'scgen'
    obsp: 'distances', 'connectivities'

In [ ]:
pred, delta = model.predict(
    ctrl_key="control", stim_key="stimulated", celltype_to_predict="CD4 T cells"
)

# we annotate the predicted cells to distinguish them later from ground truth cells.
pred.obs["condition"] = "prediction"
print(pred)

INFO     Received view of anndata, making copy.                                                                    


In [ ]:
pred.obs

In [ ]:

train = adata
ctrl_adata = adata[
    ((adata.obs["cell_type"] == "CD4 T cells") & (adata.obs["condition"] == "control"))
]
stim_adata = train[((train.obs['cell_type'] == 'CD4T') & (train.obs['condition'] == 'stimulated'))]

# concatenate pred, control and real CD4 T cells in to one object
eval_adata = ctrl_adata.concatenate(cd4t_stim, pred)

In [ ]:
sc.tl.pca(eval_adata)
sc.pl.pca(eval_adata, color="condition", frameon=False)

In [ ]:
cd4t_adata = adata[adata.obs["cell_type"] == "CD4 T cells"]

In [ ]:
sc.tl.rank_genes_groups(cd4t_adata, groupby="condition", method="wilcoxon")
diff_genes = cd4t_adata.uns["rank_genes_groups"]["names"]["stimulated"]
diff_genes

In [ ]:
r2_value = model.reg_mean_plot(
    eval_adata,
    axis_keys={"x": "predicted stimulated", "y": "stimulated"},
    gene_list=diff_genes[:10],
    top_100_genes=diff_genes,
    labels={"x": "predicted", "y": "ground truth"},
    show=True,
    legend=False,
)

In [ ]:
sc.pl.violin(eval_adata, keys="ISG15", groupby="condition")

In [ ]:
import pertpy as pt
import muon as mu
import scanpy as sc

In [ ]:
mdata = pt.dt.papalexi_2021()
for col in mdata.obs: print(col)

In [ ]:
sc.pp.normalize_total(mdata["rna"])
sc.pp.log1p(mdata["rna"])
sc.pp.highly_variable_genes(mdata["rna"], subset=True)

In [ ]:
mu.prot.pp.clr(mdata["adt"])

In [ ]:
sc.pp.pca(mdata["rna"])

In [ ]:
# We calculate neighbors with the cosine distance similarly to the original Seurat implementation
sc.pp.neighbors(mdata["rna"], metric="cosine")

In [ ]:
sc.tl.umap(mdata["rna"])

In [ ]:
sc.pl.umap(mdata["rna"], color=["replicate", "Phase", "perturbation"])

In [ ]:
print(adata_pert)

In [ ]:
# ms = pt.tl.Mixscape()

# ms.perturbation_signature(
#     mdata["rna"],
#     pert_key="perturbation",
#     control="NT",
#     split_by="replicate",
#     n_neighbors=20,
# )

In [ ]:
# We create a copy of the object to recalculate the PCA.
# Alternatively we could replace the X of the RNA part of our MuData object with the `X_pert` layer.
# adata_pert = mdata["rna"].copy()
# adata_pert.X = adata_pert.layers["X_pert"]
# sc.pp.pca(adata_pert)
# sc.pp.neighbors(adata_pert, metric="cosine")
# sc.tl.umap(adata_pert)
# sc.pl.umap(adata_pert, color=["replicate", "Phase", "perturbation"])

In [ ]:
# ms.mixscape(adata=mdata["rna"], control="NT", labels="gene_target", layer="X_pert")

In [ ]:
# mdata["rna"].obs

In [ ]:
# pt.pl.ms.perturbscore(
#     adata=mdata["rna"], labels="gene_target", target_gene="IFNGR2", color="orange"
# )

In [ ]:
# sc.settings.set_figure_params(figsize=(10, 10))
# pt.pl.ms.violin(
#     adata=mdata["rna"],
#     target_gene_idents=["NT", "IFNGR2 NP", "IFNGR2 KO"],
#     groupby="mixscape_class",
# )

In [ ]:
# pt.pl.ms.heatmap(
#     adata=mdata["rna"],
#     labels="gene_target",
#     target_gene="IFNGR2",
#     layer="X_pert",
#     control="NT",
# )

In [ ]:
# mdata["adt"].obs["mixscape_class_global"] = mdata["rna"].obs["mixscape_class_global"]
# pt.pl.ms.violin(
#     adata=mdata["adt"],
#     target_gene_idents=["NT", "JAK2", "STAT1", "IFNGR1", "IFNGR2", "IRF1"],
#     keys="PDL1",
#     groupby="gene_target",
#     hue="mixscape_class_global",
# )

In [ ]:
# ms.lda(adata=mdata["rna"], labels="gene_target", layer="X_pert")

In [ ]:
# pt.pl.ms.lda(adata=mdata["rna"])
#https://zenodo.org/record/7058382
# folders = '/home/awahab/llm-testing/data_sets/'
# fp1 = 'SC3_v3_NextGem_DI_CRISPR_A549_5K_Multiplex_count_raw_feature_bc_matrix.h5'
# fp2 = 'SC3_v3_NextGem_DI_CRISPR_A549_5K_Multiplex_count_raw_molecule_info.h5'


# import h5py
# import anndata

# # Read the .h5 File
# def explore_h5py_group(group, indent=0):
#     """Recursively print the contents of an h5py group/dataset."""
#     print(group.data.name)
#     items = sorted(group.items())
#     for name, item in items:
#         if isinstance(item, h5py.Dataset):  # Check if item is a dataset
#             print("  " * indent + f"Dataset: {name} (Shape: {item.shape}, Dtype: {item.dtype})")
#         elif isinstance(item, h5py.Group):  # Check if item is a group
#             print("  " * indent + f"Group: {name}")
#             explore_h5py_group(item, indent + 1)  # Recursive call to explore subgroups

# # Open your HDF5 file
# with h5py.File(folders + fp1, 'r') as f:
#     explore_h5py_group(f)